In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime
from collections import Counter

groupings = {
            'Medical Supplies':['medical supplies','test kits'],
            'PPE':['face mask','hand sanitizer'],
            'Recommendations':['social distancing', 'shelter in place'],
            'Restrictions':['school closure', 'travel ban'],
            'Seasonal Flu':['flu', 'common cold']
             }

__author__ = 'Haomin Lin'
__version__ = 'Python3'

In [2]:
path_info_news = '../Downloads/info_news.csv'
info_news = pd.read_csv(path_info_news,index_col=0)
# info_news = info_news[info_news['length']>100]
info_news['date'] = [datetime.strftime(pd.to_datetime(date), '%Y-%m-%d') for date in tqdm_notebook(info_news['date'])]

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
path_info_blog = '../Downloads/info_blogs.csv'
info_blog = pd.read_csv(path_info_blog,index_col=0)
# # info_blog = info_blog[info_blog['length']>100]
info_blog['date'] = [datetime.strftime(pd.to_datetime(date), '%Y-%m-%d') for date in tqdm_notebook(info_blog['date'])]

In [4]:
path_info_disc = '../Downloads/info_discussions.csv'
info_disc = pd.read_csv(path_info_disc,index_col=0)
# # info_disc = info_disc[info_disc['length']>100]
info_disc['date'] = [datetime.strftime(pd.to_datetime(date), '%Y-%m-%d') for date in tqdm_notebook(info_disc['date'])]

In [5]:
# Select countries common in all 3 data sets
country_map = Counter(set(info_news['country']))+Counter(set(info_blog['country']))+ Counter(set(info_disc['country']))
common_locations = []
for k,v in tqdm_notebook(country_map.items()):
    if (v == 3) and (k==k):
        common_locations.append(k)
print("Common Countries:", len(common_locations))


Common Countries: 79


In [6]:
# Filter data by common countries
info_news = info_news[info_news['country'].isin(common_locations)]
info_blog = info_blog[info_blog['country'].isin(common_locations)]
info_disc = info_disc[info_disc['country'].isin(common_locations)]
print ("News:", len(info_news))
print ("Blogs:", len(info_blog))
print ("Discussions:", len(info_disc))

News: 2277563
Blogs: 946755
Discussions: 409091


In [7]:
all_results = []
for platform,p in zip([info_news,info_blog,info_disc],['News','Blogs','Discussions']):
    
    platform_themes=[]
    for theme,topics in tqdm_notebook(groupings.items()):
        theme_df = platform[(platform['topic'].str.contains(topics[0]))|(platform['topic'].str.contains(topics[1]))]
        es = {key: np.true_divide(Counter(theme_df['date'])[key], Counter(platform['date']).get(key, 0)) for key in Counter(theme_df['date']).keys()}
        theme_df = theme_df[['date','theme']].groupby('date').agg('max').reset_index()
        theme_df['theme']=[theme]*len(theme_df)
        theme_df['topic_share'] = [es.get(date) for date in theme_df['date']] 
        theme_df['platform'] = [p]*len(theme_df)
        theme_df['date_2'] = theme_df['date']
        theme_df['topic_share_2'] = theme_df['topic_share']
        platform_themes.append(theme_df)
        
    all_results.append(pd.concat(platform_themes))
tableau_df = pd.concat(all_results)
tableau_df.to_csv('tableau_df.csv',index=False)
tableau_df.head(10)

,date,theme,topic_share,platform,date_2,topic_share_2
0,2019-12-03,Medical Supplies,0.333333,News,2019-12-03,0.333333
1,2019-12-17,Medical Supplies,0.333333,News,2019-12-17,0.333333
2,2020-01-05,Medical Supplies,0.015385,News,2020-01-05,0.015385
3,2020-01-08,Medical Supplies,0.017241,News,2020-01-08,0.017241
4,2020-01-10,Medical Supplies,0.005128,News,2020-01-10,0.005128
5,2020-01-11,Medical Supplies,0.004098,News,2020-01-11,0.004098
6,2020-01-14,Medical Supplies,0.011834,News,2020-01-14,0.011834
7,2020-01-15,Medical Supplies,0.003984,News,2020-01-15,0.003984
8,2020-01-16,Medical Supplies,0.003145,News,2020-01-16,0.003145
9,2020-01-18,Medical Supplies,0.005338,News,2020-01-18,0.005338
